# Pattern Augmentation - Policy

In [1]:
import InspectorG.policy_aug as pa

"""
Builtin operations in InspectorG.policy_aug
"""
opers = [ 
        (pa.ShearX, -0.3, 0.3), 
        (pa.ShearY, -0.3, 0.3),
        (pa.TranslateX, -0.45, 0.45),
        (pa.TranslateY, -0.45, 0.45),
        (pa.AutoContrast, 0, 1),
        (pa.Rotate, -30, 30),
        (pa.Invert, 0, 1),
        (pa.Equalize, 0, 1),
        (pa.Solarize, 0, 256),
        (pa.Posterize, 4, 8),
        (pa.Contrast, 0.1, 1.9),
        (pa.Color, 0.1, 1.9),
        (pa.Brightness, 0.1, 1.9),
        (pa.Sharpness, 0.1, 1.9),
        (pa.Cutout, 0, 0.2),
        (pa.ResizeX, 0.8, 1.2),
        (pa.ResizeY, 0.8, 1.2)
    ]

"""
User-defined operation
"""
def RotateNew(img, v):  # [-30, 30]
    return img.rotate(v)

opers += (RotateNew, -20, 20)
oper_idxs = [1, 3, 4, 13]


DTYPE = 'KSDD'
p_path = './InspectorG/PATTERN/'+ DTYPE
aug_dir = p_path + '-policy-tmp/' # or p_path + '-policy/'
img_typ = 'png'

pa.PatternAugPolicy(DTYPE, opers, oper_idxs = oper_idxs, p_path = p_path, aug_dir = aug_dir, img_typ = img_typ)

# Pattern Augmentation - GAN

### gan_aug_trainer.py

#### PARAMETER

--n_epochs 	: Number of epochs (default : 200)\
--batch_size 	: Mini-batch size (default : 1)\
--vector_size	: The size of input random noise vector (default : 100)\
--lr_G		: Learning rate of Geneartor	(default : 0.0001)\
--lr_D		: Learning rate of Discriminator (default : 0.0001)\
--d_image	: Dimension of input image (color : 3, grayscale : 1)\
--resize		: If every input images have different sizes, image resize is required. The size of resize image (default : 100) * If the resize is changed, DISCRIMNATOR and GENERATOR are also changed.\
--path		: Folder which has images\
--c_period	: Checkpoint period. Every c_period epochs, backup is executed. (default : 100)\ 

##### Ex)  python gan_aug_trainer.py --batch_size 5 --d_image 1 --path NEW_IG/SKT_half/Scratch/Scratch/ --resize 100 --n_epochs 2000 --c_period 100

### gan_aug_generator.py

#### PARAMETER

--vector size\
--d_image\
--image_size 	: Image_size same as resize or original image size used at training GAN\
--save_path	: Path that generated images are saved\
--save_folder_path	: Path of save_folder (the folder of package)\
--model_path	: Path of generator model (pkl file)\
--g_num		: The number of generated images\

##### Ex) python gan_aug_generator.py --vector_size 100 --d_image 1 --save_path Generated_image/ --save_folder_path Save_folder/ --model_path Save_folder/G_Model_Save/Generator.pkl --g_num 10


# Feature Generation

In [2]:
from InspectorG.utils import SliceBbox, ImgViewer, SaveDict, LoadDict
from InspectorG.feature_generation import FeatureGenerator, FeatureManager

DTYPE = 'KSDD'

IMGDICT = LoadDict('Experiment','imgdict')[DTYPE]
DEVDICT = LoadDict('DEVELOPMENT')[DTYPE]

In [3]:
""" 
Image data path Dictionary

{Product ID : (PATH, Label)}
"""
IMGDICT

{'46': ('../Dataset/KolektorSDD/kos12/Part4.jpg', 0),
 '303': ('../Dataset/KolektorSDD/kos16/Part0.jpg', 0),
 '102': ('../Dataset/KolektorSDD/kos28/Part4.jpg', 0),
 '68': ('../Dataset/KolektorSDD/kos31/Part7.jpg', 0),
 '12': ('../Dataset/KolektorSDD/kos11/Part7.jpg', 0),
 '287': ('../Dataset/KolektorSDD/kos38/Part0.jpg', 1),
 '186': ('../Dataset/KolektorSDD/kos05/Part1.jpg', 0),
 '44': ('../Dataset/KolektorSDD/kos12/Part7.jpg', 0),
 '32': ('../Dataset/KolektorSDD/kos02/Part0.jpg', 0),
 '311': ('../Dataset/KolektorSDD/kos13/Part0.jpg', 0),
 '191': ('../Dataset/KolektorSDD/kos45/Part0.jpg', 0),
 '289': ('../Dataset/KolektorSDD/kos38/Part3.jpg', 0),
 '235': ('../Dataset/KolektorSDD/kos18/Part7.jpg', 0),
 '177': ('../Dataset/KolektorSDD/kos14/Part3.jpg', 0),
 '305': ('../Dataset/KolektorSDD/kos16/Part3.jpg', 0),
 '372': ('../Dataset/KolektorSDD/kos29/Part6.jpg', 0),
 '257': ('../Dataset/KolektorSDD/kos01/Part3.jpg', 0),
 '338': ('../Dataset/KolektorSDD/kos49/Part1.jpg', 0),
 '231': ('../Da

In [4]:
""" 
Development set Dictionary

{Product ID : Label}
"""
DEVDICT

{'47': 1,
 '68': 0,
 '113': 0,
 '295': 0,
 '363': 0,
 '60': 0,
 '12': 0,
 '48': 1,
 '259': 0,
 '321': 0,
 '49': 0,
 '198': 0,
 '37': 1,
 '200': 0,
 '26': 1,
 '111': 0,
 '210': 0,
 '373': 0,
 '203': 0,
 '140': 0,
 '247': 0,
 '294': 0,
 '362': 0,
 '285': 0,
 '10': 0,
 '205': 0,
 '386': 0,
 '255': 0,
 '81': 0,
 '155': 0,
 '59': 1,
 '62': 0,
 '398': 0,
 '5': 0,
 '77': 0,
 '105': 0,
 '28': 0,
 '300': 0,
 '38': 0,
 '27': 0,
 '67': 1,
 '116': 0,
 '370': 0,
 '41': 0,
 '243': 0,
 '1': 1,
 '119': 0,
 '279': 0,
 '217': 0,
 '204': 0,
 '14': 1,
 '35': 0,
 '21': 1,
 '329': 0,
 '163': 0,
 '30': 0,
 '213': 0,
 '75': 1,
 '53': 0,
 '284': 0,
 '141': 0,
 '118': 0,
 '170': 0,
 '376': 0,
 '100': 0,
 '328': 0,
 '393': 0,
 '356': 0,
 '42': 0,
 '293': 0,
 '158': 0,
 '138': 0,
 '291': 0,
 '58': 0,
 '64': 0,
 '251': 0,
 '239': 0,
 '79': 0}

In [5]:
FeatureE_org = FeatureGenerator(imgdict = IMGDICT, task_name = DTYPE)
FeatureE_GAN = FeatureGenerator(imgdict = IMGDICT, task_name = DTYPE, aug = 'GAN')
FeatureE_policy = FeatureGenerator(imgdict = IMGDICT, task_name = DTYPE, aug = 'policy')

FeatureE_org.GenFeature(save = True)
FeatureE_GAN.GenFeature(save = True)
FeatureE_policy.GenFeature(save = True)

100%|██████████| 100/100 [00:00<00:00, 9158.87it/s]

Generate Features ...
67_a_1 -> Already Generated
14_a_1 -> Already Generated
21_a_1 -> Already Generated
59_a_1 -> Already Generated
26_a_1 -> Already Generated
37_a_1 -> Already Generated
48_a_1 -> Already Generated
47_a_1 -> Already Generated
75_a_1 -> Already Generated
1_a_1 -> Already Generated
Generate Features ...
KSDD_84 -> Already Generated
KSDD_74 -> Already Generated
KSDD_5 -> Already Generated
KSDD_52 -> Already Generated
KSDD_44 -> Already Generated
KSDD_89 -> Already Generated
KSDD_92 -> Already Generated
KSDD_91 -> Already Generated
KSDD_53 -> Already Generated
KSDD_0 -> Already Generated
KSDD_3 -> Already Generated
KSDD_70 -> Already Generated
KSDD_12 -> Already Generated
KSDD_79 -> Already Generated
KSDD_64 -> Already Generated
KSDD_93 -> Already Generated
KSDD_69 -> Already Generated
KSDD_58 -> Already Generated
KSDD_32 -> Already Generated
KSDD_66 -> Already Generated
KSDD_2 -> Already Generated
KSDD_78 -> Already Generated
KSDD_9 -> Already Generated
KSDD_54 -> Alre

In [6]:
""" 
Feature Dictionary

{Product ID : {Pattern Name : (Confidence, Bounding Box)}}
"""
FeatureE_org.LoadFeatureDict(wo_bbox = False) # wo_bbox -> without bonunding box 

{'46': {'26_a_1': (0.2572319507598877, (0, 813, 440, 866)),
  '75_a_1': (0.23772113025188446, (76, 905, 442, 992)),
  '48_a_1': (0.41756901144981384, (0, 380, 315, 421)),
  '47_a_1': (0.39897793531417847, (166, 82, 323, 146)),
  '21_a_1': (0.3146161139011383, (86, 639, 434, 685)),
  '59_a_1': (0.49781182408332825, (111, 327, 303, 352)),
  '14_a_1': (0.5135915875434875, (193, 1208, 441, 1230)),
  '37_a_1': (0.5281617045402527, (306, 563, 438, 590)),
  '67_a_1': (0.4808754622936249, (140, 579, 500, 620)),
  '1_a_1': (0.49827080965042114, (266, 478, 498, 577))},
 '369': {'26_a_1': (0.288124144077301, (39, 105, 479, 158)),
  '75_a_1': (0.24290725588798523, (56, 461, 422, 548)),
  '48_a_1': (0.39415213465690613, (136, 1098, 451, 1139)),
  '47_a_1': (0.3854585587978363, (318, 45, 475, 109)),
  '21_a_1': (0.36067983508110046, (122, 122, 470, 168)),
  '59_a_1': (0.5310207009315491, (11, 1206, 203, 1231)),
  '14_a_1': (0.48669326305389404, (154, 459, 402, 481)),
  '37_a_1': (0.5923135876655579,

In [7]:
FeatureM = FeatureManager(task_name = DTYPE, f_org = FeatureE_org, f_gan = FeatureE_GAN, f_policy = FeatureE_policy)

"""
Choose one mode in (org, gan, policy, all)

pat_num : Number of Patterns
"""

MODE = 'org'
X_tr, Y_tr, X_te, Y_te = FeatureM.MakeMatrix(dev_dict = DEVDICT, mode = MODE, pat_num = 100)

MODE : org

== PATTENRNS ==
ORG : 10, GAN : 0, Policy : 0

Make Training Data ...
Train : (78, 10), (78,), Test : (321, 10), (321,)


In [8]:
MODE = 'gan'
X_tr, Y_tr, X_te, Y_te = FeatureM.MakeMatrix(dev_dict = DEVDICT, mode = MODE, pat_num = 100)

MODE : gan

== PATTENRNS ==
ORG : 10, GAN : 100, Policy : 0

Make Training Data ...
Train : (78, 110), (78,), Test : (321, 110), (321,)


In [9]:
MODE = 'policy'
X_tr, Y_tr, X_te, Y_te = FeatureM.MakeMatrix(dev_dict = DEVDICT, mode = MODE, pat_num = 100)

MODE : policy

== PATTENRNS ==
ORG : 10, GAN : 0, Policy : 100

Make Training Data ...
Train : (78, 110), (78,), Test : (321, 110), (321,)


In [10]:
MODE = 'all'
X_tr, Y_tr, X_te, Y_te = FeatureM.MakeMatrix(dev_dict = DEVDICT, mode = MODE, pat_num = 100)

MODE : all

== PATTENRNS ==
ORG : 10, GAN : 100, Policy : 100

Make Training Data ...
Train : (78, 210), (78,), Test : (321, 210), (321,)


In [11]:
MODE = 'all'
"""
To use all patterns => pat_num = None
"""
X_tr, Y_tr, X_te, Y_te = FeatureM.MakeMatrix(dev_dict = DEVDICT, mode = MODE, pat_num = None)

MODE : all

== PATTENRNS ==
ORG : 10, GAN : 100, Policy : 100

Make Training Data ...
Train : (78, 210), (78,), Test : (321, 210), (321,)


# Labeling

In [12]:
from InspectorG import labeler

def train_func(xtr, ytr):
    defectnum = sum(ytr)
    for i in range(1,10):
        if defectnum/i < 10:
            k = min(max(i-1, 1), 3)
            break
    return labeler.train_recursive_cv(xtr, ytr, k, 3, test_iter = 20, rand = 17)

Model = train_func(X_tr, Y_tr)

Number of Defective Data : 10, K (Cross Validation) : 1


/home/geon/anaconda3/envs/snorkel/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Searching - complete


In [13]:
import sklearn.metrics

def print_result(true, pred, score):
    cls_rpt = sklearn.metrics.classification_report(true, pred, target_names=['ok', 'defective'], digits=4)
    print(cls_rpt)
    rocauc = sklearn.metrics.roc_auc_score(true, score)
    ap = sklearn.metrics.average_precision_score(true, score)
    acc = sklearn.metrics.accuracy_score(true, pred)
    print('ACC : %s\nROC - AUC : %s\nAVERAGE - PRECISION : %s' % (acc, rocauc, ap))
    return rocauc, ap, cls_rpt

def evaluate_model(model, xte, yte):
    print('\nEvaluate Model ...')
    ypred, yscore = model.predict(xte), model.predict_proba(xte)[:, 1]
    return print_result(yte, ypred, yscore)

def model_labeling(model, xte):
    return model.predict(xte), model.predict_proba(xte)[:, 1]

_, _, cls_rpt = evaluate_model(Model, X_te, Y_te)
Y_pred = model_labeling(Model, X_te)


Evaluate Model ...
             precision    recall  f1-score   support

         ok     0.9331    1.0000    0.9654       279
  defective     1.0000    0.5238    0.6875        42

avg / total     0.9419    0.9377    0.9290       321

ACC : 0.9376947040498442
ROC - AUC : 0.9272060078511691
AVERAGE - PRECISION : 0.809192089448623
